# The `Structures` notebook  <a id="Structures"></a>[<font size=1>(back to `Main.ipynb`)</font>](./Main.ipynb)

This notebook gathers all structures -- and their related constructors -- that are used in the computation of the steady-state Ramsey allocation.

### The notebook is organized as follows:

1. the [`Economy` structure](#economy-struct) characterizing the economy parameters (computational input);
2. the [`AiyagariSolution` structure](#Aiyagari) containing the characterization of the Aiyagari equilibrium, such as policy functions and stationary distribution (computational output). 

# The `Economy` structure <a id="economy"></a>[<font size=1>(back to menu)</font>](#Structures)

This is a non-mutable structure characterizing the economy parameters. It is used as input all most of our functions. We proceed in two steps:

1. the [structure definition](#economy-struct),
2. a [specific constructor](#economy-const) performing calibration.

## Structure definition <a id="economy-struct"></a>[<font size=1>(back to `Economy`)</font>](#economy)

There are four blocks. Note that in most cases the implementation of the variable is direct and hence not specified. If any doubt, the implementation is given. For example, $\beta$ is `β` (so not specified), while the grid $a_{Grid}$ is `aGrid` is specified. 
1. Economy parameters: 
* *Utility function related parameters.* They are: discount factor $\beta$, inverse of constant IES $\gamma$. The intertemporal utility is $\sum_{t=0}^\infty \beta^t U(c_t)$ with $U(c) = \frac{c^{1-\gamma}}{1-\gamma} $.
> ***Remark.*** Julia allows for specifying functions in struct (see below), so keeping utility parameters ($\gamma$, $\varphi$, and $\chi$) after calibration is not mandatory. Actually they are never used in the Julia code. However, they are needed in Dynare which does not easily allow to pass functions from Julia. This is why they are kept.

2. *Utility functions and derivatives.* For consumption: $u$ (defined as $u(c)=\frac{c^{1-\gamma}}{1-\gamma}$), $u^\prime$, the inverse of $u^\prime$ denoted by $u^{\prime -1}$ (`inv_u′`), $u^{\prime\prime}$. 

3. *The asset grid and some of its characteristics.* The asset grid $a_{Grid}$ (`aGrid`) is a vector whose elements are denoted by $a_{Grid,i_a}$. Its other charcteristics are: length $n_a$ (`na`), minimal element $a_\min$ (`amin`), and maximal element $a_\max$ (`amax`). These three elements can easily be recovered from the asset grid vector but are kept for simplicity sake. 

4. *The productivity process.* We allow for severance periods and severance pay (see the constructor example [below](#another_cell) for further details). The productivity process is a first-order Markov chain with the following characteristics. 
    * `y_size_raw`: nb of income levels (excluding severance periods and severance pay)
    * `times_to_u`: nb of severance periods, which is equal to the nb of periods before falling into unemployment (once learning it)
    * `y_size`: actual nb of income levels that includes the states corresponding to severance periods (i.e.,`y_size` $=1+($ `y_size_raw` $-1)\times(1+$ `times_to_u`$)$).
    * `Πy` transition matrix of size `y_size`$\times$ `y_size`
    * `S_y` stationary distribution ($\sum_y S_y Π_{y y^\prime} = S_{y^\prime}$ and $\sum_y S_y=1$).
    * `y_vals` vector of income levels (of size  `y_size`), that possibility includes severance pay.

    The transition matrix is $\Pi^y$ (`Πy`) and the stationary distribution is given by the vector $S_y$ (`Sy`): `Πy[1,2]` is the transition probability from `ys[1]` to `ys[2]`, while `Sy[2]` is the stationary size of agents with producitivity `ys[2]`.

In [1]:
struct Economy{T<:Real,I<:Int,T1<:Function,T2<:Function,T3<:Function,T4<:Function,T5<:Function,T6<:Function,
                T7<:Function,T8<:Function}
    # preference parameters
    β::T                 # discount factor
    γ::T
    χ::T
    φ::T
    u::T1                # per-period utility function
    u′::T2               # per-period marginal utility function
    inv_u′::T3           # inverse of u′
    ψ::T4                # period effort disutility
    ψ′::T5                # period effort disutility
    inv_ψ′::T6                # period effort disutility
    
    
    # defining asset grid for model solution: a ∈ aGrid is end of period assets
    a_size::I         # number of asset grid points
    a_min ::T         # min asset holdings
    a_max ::T         # max asset holdings    
    aGrid ::Vector{T} # grid for asset choices
    
    # income process
    y_size_raw::I         # nb of income levels (excluding severance pay and periods)
    times_to_u::I         # time to unemployment (nb of severance periods)
    y_size::I             # actual nb of income states, including severance states (=1+(y_size_raw-1)*(1+times_to_u))
    Πy::T7                # transition matrix
    Sy::T8                # stationary distribution (Sy Πy = Sy and sum(Sy)=1)
    ys::Vector{T}         # vector of of all income states (of size y_size)
    ie::I                 # index of the employed productivity (ys[ie] is the employed productivity level)
    iu::I                 # index of the unemployed productivity (ys[iu] is the unemployed productivity level)
    

    # prices and costs
    R::T                 # gross post tax interest rate
    w::T                 # wage rate
    Tt::T                # transfers
end

## Calibration via constructors <a id="economy-const"></a>[<font size=1>(back to `Economy`)</font>](#economy)

The constructors below calibrate the economy parameters based on a number of inputs.
1. Utility functions are defined using $\gamma$.
2. Besides asset and productivity parameters, all other parameters are exogenously set.
3. For the asset grid, we define an exponential grid with curvature `curv_a ` and `na` elements between `amin` and `amax`. We divide the grid by $1+\tau^c$ to account for the presence of consumption tax.
4. For the productivity process, we  add to all states (but the worst) `times_to_u` periods before transition to the worst state. Note that with our procedure the worst state corresponds to index 1. Formally, we add `times_to_u` states and the transition is deterministic from one state to another until unemployment is reached. Finally, the last payment before worst state is inscreased by `severance_pay`%.

We consider two consructors, one in which an AR(1) process is discretized by a Rouwenhorst procedure and another one in which the inputs are a vector of replacement rate and two transition probabilities. 

<a id='another_cell'></a>**Example for the income process** (we use the numerical values of the [computational example](#eco-example) below).

Assume that the Rouwenhorst procedure implies a transition matrix 
$$\tilde T = \left(\begin{array}{cc} 0.98 & 0.02 \\ 0.02 & 0.98 \end{array}\right),$$
and an income vector:
$$\tilde y = \left(0.6573, 1.3427\right).$$

We add 3 periods before unemployment (worst state interpreted as unemployment)  and a severance pay of 25%. We thus insert 3 states between unemployment (state 1 that remains state 1) and employment (former state 2 that will become new state 5). We thus obtain the new income vector that will denoted by $\mathcal Y$:

$$\mathcal Y = \left(0.6573, 1.6784, 1.3427, 1.3427, 1.3427\right),$$

where the incomes 2 and 3 periods before unemployment remain equal to the one in employment and the income before unemployment is 25% higher. 

The transition matrix becomes:
$$T = \left(\begin{array}{cc} 
0.98 & 0 & 0 & 0 & 0.02 \\
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0.02 & 0.98 \\
\end{array}\right).$$

> ***Remark.*** There is one main calibration-constructor. There is also a secondary version with keywords calling the previous one.  

### Castastrophic states, path of replacement rates and transition probas as inputs
There are now 3 states: employed & unemployed + catastophic states

In [4]:
function mybound(x::T;upperbound::T=one(T), lowerbound::T=zero(T))::T where T<:Number
    return max(lowerbound,min(upperbound,x))
end
mybound(1.1)

1.0

In [ ]:
function Economy(
    # preference parameters
    β::T,
    γ::T,
    χ::T,
    φ::T,
    
    # asset grid
    a_size::I,    
    a_min::T, 
    a_max::T,     
    curv_a::T,
    
    # income process    
    times_to_u::I,          # nb of periods before actually falling to state 1 (unemployment) 
    path_to_u::Vector{T},    # extra payment (as a share of normal payment) made at the last month before unemployment
    πeu::T,                 # proba to remain employed
    πec::T,
    πuc::T,
    πce::T,
    πcu::T,
    yc::T,
        
    # prices
    R::T,                   # gross post tax interest rate
    w::T,                   # default value for wage
    Tt::T
    ) where {T<:Real,I<:Int}
    #=
        *** utility functions ***
    =#
    u      = c::T  -> (γ ≈ one(T)) ? log(c) : c^(one(T)-γ)/(one(T)-γ)::T
    u′     = c::T  -> c^(-γ)::T
    inv_u′ = up::T -> up^(-one(T)/γ)::T
    ψ      = x::T  -> (1/χ)*x^(one(T)+1/φ)/(one(T)+1/φ)::T
    ψ′     = x::T  -> (1/χ)*x^(1/φ)::T
    inv_ψ′ = U::T -> (χ*U)^φ::T 
    
    #=
        *** grid for asset choices ***
    =#    
    aGrid = a_min .+ (a_max-a_min)*(range(0.0, 1.0, length=a_size)).^curv_a
    
    #=
        *** productivity process ***
    =#
    #=
    πuu = mybound(πuu)
    πee = mybound(πee)
    πuc = mybound(πuc)
    πec = mybound(πec)
    πce = mybound(πce)
    πcu = mybound(πcu)=#
    y_size_raw = 3
    y_size::I = y_size_raw + times_to_u
        
    function Πy(πue)
        if πue+πuc > one(πue)
            πuc = one(πue)-πue
        end
        if πeu+πec > one(πue)
            πec = one(πue)-πeu
        end
        if πcu+πce > one(πue)
            if πce > πcu
                πce = one(πue)-πcu
            else
                πcu = one(πue)-πce
            end
        end
        #@show πuc
        Πy′::Matrix{T}         = [one(πue)-πue-πuc πue πuc;πeu one(πue)-πeu-πec πec;πcu πce one(πue)-πcu-πce]
        Πy′′::Matrix{T}          = zeros(T,y_size,y_size)
        Πy′′[1,1]     = Πy′[1,1]
        Πy′′[1,end-1] = Πy′[1,end-1]
        Πy′′[end,1]   = Πy′[end,1]
        Πy′′[1,end]   = Πy′[1,end]
        Πy′′[end-1,end]= Πy′[end-1,end]
        Πy′′[end,end-1]= Πy′[end,end-1]
        Πy′′[end,end] = Πy′[end,end]    
        Πy′′[2,1] = one(T)
        Πy′′[2+times_to_u,1+times_to_u]= Πy′[2,1]
        for t=2:times_to_u
            Πy′′[1+t,1+t-1] = one(T)
        end
        Πy′′[2+times_to_u,2+times_to_u]= Πy′[2,2]
        return Πy′′
    end
    
    y_vals′′::Vector{T}     = ones(T,y_size)
    y_vals′′[1:length(path_to_u)] .*= 1.0 .+ path_to_u
    y_vals′′[end] = yc
    
    iu = 1
    ie = y_size-1
    
    ys = y_vals′′
    
    function Sy(πue)
        Sy_ = (Πy(πue)^100_000)[1,:]    
        return Sy_./sum(Sy_)
    end
    
    
    return Economy{T,I,typeof(u),typeof(u′),typeof(inv_u′),
                       typeof(ψ),typeof(ψ′),typeof(inv_ψ′),
                       typeof(Πy),typeof(Sy)}(
            β,γ,χ,φ,
            u,u′,inv_u′,
            ψ,ψ′,inv_ψ′,
            a_size,a_min,a_max,aGrid,
            y_size_raw,times_to_u,y_size,Πy,Sy,ys,ie,iu,
            R,w,Tt)
end;

function Economy(;
    # preference parameters
    β::T,
    γ::T,
    χ::T,
    φ::T,
    
    # asset grid
    a_size::I,    
    a_min::T, 
    a_max::T,     
    curv_a::T,
    
    # income process
    times_to_u::I,             # nb of periods before actually falling to state 1 (unemployment) 
    path_to_u::Vector{T},    # extra payment (as a share of normal payment) made at the last month before unemployment
    πeu::T,                 # proba to remain employed
    πec::T,
    πuc::T,
    πce::T,
    πcu::T,
    yc::T,
        
    # prices
    R::T,                   # gross post tax interest rate
    w::T,                   # default value for wage
    Tt::T
    ) where {T<:Real,I<:Int}
    return Economy(β,γ,χ,φ,a_size,a_min,a_max,curv_a,times_to_u,path_to_u,πeu,πec,πuc,πce,πcu,yc,R,w,Tt)
end;

# The `AiyagariSolution` structure <a id="Aiyagari"></a>[<font size=1>(back to menu)</font>](#Structures)

This is a *mutable* structure containing the characterization of the Aiyagari equilibrium, such as policy functions and stationary distribution It is used an output of our computations. The Aiyagari equilibrium is computed in [`SolveAiyagari.ipynb`](./SolveAiyagari.ipynb). We proceed in two steps:

1. the [structure definition](#Aiyagari-struct),
2. a [specific constructor](#Aiyagari-const) performing calibration.

## Structure definition <a id="Aiyagari-struct"></a>[<font size=1>(back to `AiyagariSolution`)</font>](#Aiyagari)

The structure contains the following elements.
* Policy functions: $g_a$ (`ga`), $g_c$ (`ga`), $g_l$ (`ga`) for savings, consumption and labor supply respectively. They are implemented as $n_a \times n_y$ matrices.
* Interest and wage rates: $R$ (`R`) and $w$ (`w`).
* Aggregate quantities: aggregate savings $A$, aggregate consumption $C$, aggregate labor supply $L$,  output $Y$.
* Transition matrix `transitMat` on the product grid (asset $\times$ productivity) of size $n_a\cdot n_y \times n_a\cdot n_y$.
* The stationary distribution `stationaryDist` associated to `transitMat`. It is implemented not as a $n_a\cdot n_y$ vector but a $n_a\times n_y$ matrix (as policy functions), which makes the link with the usual recursive approach simpler. 
* The residuals of the individual Euler equations, `residEuler` as a $n_a\times n_y$ matrix.

In [5]:
mutable struct AiyagariSolution{T<:Real,I<:Integer}    
    ga::Matrix{T}                    # policy function for savings on the asset grid
    gl::Matrix{T}                    # policy function for labor on the asset grid
    gc::Matrix{T}                    # policy function for consumption on the asset grid
    gf::Matrix{T}                    # policy function for effort choice on the asset grid
    Us::Matrix{T}                    # value function on the asset grid 
    R::T                             # gross interest rate
    w::T                             # wage rate
    A::T                             # aggregate savings
    C::T                             # aggregate consumption
    L::T                             # aggregate labor supply (in efficient units)
    F::T
    transitMat::SparseMatrixCSC{T,I} # transition matrix
    stationaryDist::Matrix{T}        # stationary distribution
    residEuler::Matrix{T}            # residual of Euler equations
end

## The constructor <a id="Aiyagari-const"></a>[<font size=1>(back to `AiyagariSolution`)</font>](#Aiyagari)

The constructor is invoked by:
>`AiyagariSolution(eco::Economy)`.

The quantities are filled very basically and this does not matter much. The only point worth mentioning is about interest rates and wages that are copied from the [`Economy` element](#economy-const).

In [8]:
function AiyagariSolution(eco::Economy{T,I,T1,T2,T3,T4,T5,T6,T7,T8}) where {T<:Real,
            T1<:Function,T2<:Function,T3<:Function,T4<:Function,T5<:Function,
            T6<:Function,T7<:Function,T8<:Function,I<:Int64}
    @unpack β,γ,ys,u,ψ,a_size,u′,y_size,ys,aGrid,R,w,iu = eco
        
    cs = repeat(ys', outer=(a_size,1))
    
    ls = ones(a_size,y_size)
    as = repeat(aGrid, outer=(1,y_size))
    fs = zeros(a_size,y_size)
    fs[:,iu] .= 0.05
    Us = zeros(size(as))
    @assert size(cs) == size(ls) == size(as) == (a_size,y_size)
    L  = sum(@. ys*ls[1,:])
    A  = sum(as)
    F  = 0.05
    return AiyagariSolution{T,I}(
        as,ls,cs,fs,Us,R,w,A,A,L,F,
        spzeros(T, I, a_size*y_size,a_size*y_size), 
        fill(one(T)/(a_size*y_size), a_size,y_size), zeros(T,a_size,y_size))
end;

In [3]:
T = rand(2,6)
S = zeros(size(T))

2×6 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0